In [19]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS 

embeddings = OllamaEmbeddings(
    model="llama3.2",
)


def retrieve_apache_embeddings(level:str, content : str):     

    vectorstore = FAISS.load_local("apache_vector_store", embeddings, allow_dangerous_deserialization=True)
    print("inside event", level, content)
    query = f'''
    Retrieve Event Id and Event Template from this error log 
    {level} {content}
    '''

    docs = vectorstore.similarity_search_with_relevance_scores(query,fetch_k=4)

    return [ docs[0][0].page_content , level , content]

def retrieve_zookeeper_embeddings(level:str, component:str, content : str):     

    vectorstore = FAISS.load_local("zookeeper_vector_store", embeddings, allow_dangerous_deserialization=True)
    print("inside event", level, content)
    query = f'''
    Retrieve Event Id and Event Template from this error log 
    {level} {component} {content}
    '''

    docs = vectorstore.similarity_search_with_relevance_scores(query,fetch_k=4)

    return [ docs[0][0].page_content , level , content]


def retrieve_linux_embeddings(level:str, component:str, content : str):     

    vectorstore = FAISS.load_local("linux_vector_store", embeddings, allow_dangerous_deserialization=True)
    print("inside event", level, content)
    query = f'''
    Retrieve Event Id and Event Template from this error log 
    {level} {component} {content}
    '''

    docs = vectorstore.similarity_search_with_relevance_scores(query,fetch_k=4)

    return [ docs[0][0].page_content , level , content]
 

In [20]:
import typing as t
from pydantic import BaseModel, Field
from langchain_core.tools import BaseTool, tool
from langchain_core.tools import StructuredTool


class DebugApacheErrorLog(BaseModel):
    """Request schema for Debug Apache Error Log"""

    Level: str = Field( 
        ...,
        alias="Level",
        description="Provides severity level about the error log",        
    ) 

    Content: str = Field( 
        ...,
        alias="Content",
        description="Shows the error message of the event", 
    )
    

def debug_apache_error_log(Level:str,  Content:str):
        """ Read a debug Apache Error Log and return 
        Tool that retrieves solution from the existing logs if found, else creates a new solution
        
        Args:
            Level: Provides severity level about the error log 
            Content: Shows the error message of the event
        """
    
        print("log found", Content , Level)
        if  Content:
            output = retrieve_apache_embeddings(Level, Content)
            return output
        else:
            raise "Unknown Error"

debug_apache_error_log = StructuredTool.from_function(
    func=debug_apache_error_log,
    name="Debug Apache Error",
    description="Read a debug Apache Error Log and return",
    args_schema=DebugApacheErrorLog, 
    # coroutine= ... <- you can specify an async method if desired as well
) 

In [21]:
class DebugZooKeeperErrorLog(BaseModel):
    """Request schema for Debug ZooKeeper Error Log"""

    Level: str = Field( 
        ...,
        alias="Level",
        description="Provides severity level about the error log",        
    ) 

    Component: str = Field( 
        ...,
        alias="Component",
        description="Shows the which component throws the error", 
    )

    Content: str = Field( 
        ...,
        alias="Content",
        description="Shows the error message of the event", 
    )
    

def debug_zookeeper_error_log(Level:str, Component:str,  Content:str):
        """ Read a debug Zookeeper Error Log and return 
        Tool that retrieves solution from the existing logs if found, else creates a new solution
        
        Args:
            Level: Provides severity level about the error log 
            Component: Shows the which component throws the error
            Content: Shows the error message of the event
        """
    
        print("log found", Content , Level)
        if  Content:
            output = retrieve_zookeeper_embeddings(Level,Component, Content)
            return output
        else:
            raise "Unknown Error"

debug_zookeeper_error_log = StructuredTool.from_function(
    func=debug_zookeeper_error_log,
    name="Debug Zookeeper Error",
    description="Read a debug Zookeeper Error Log and return",
    args_schema=DebugZooKeeperErrorLog, 
    # coroutine= ... <- you can specify an async method if desired as well
) 

In [22]:
class DebugLinuxErrorLog(BaseModel):
    """Request schema for Debug ZooKeeper Error Log"""

    Level: str = Field( 
        ...,
        alias="Level",
        description="Provides severity level about the error log",        
    ) 

    Component: str = Field( 
        ...,
        alias="Component",
        description="Shows the which component throws the error", 
    )

    Content: str = Field( 
        ...,
        alias="Content",
        description="Shows the error message of the event", 
    )
    

def debug_linux_error_log(Level:str, Component:str,  Content:str):
        """ Read a debug Linux Error Log and return 
        Tool that retrieves solution from the existing logs if found, else creates a new solution
        
        Args:
            Level: Provides severity level about the error log 
            Component: Shows the which component throws the error
            Content: Shows the error message of the event
        """
    
        print("log found", Content , Level)
        if  Content:
            output = retrieve_zookeeper_embeddings(Level,Component, Content)
            return output
        else:
            raise "Unknown Error"

debug_linux_error_log = StructuredTool.from_function(
    func=debug_linux_error_log,
    name="Debug Linux Error",
    description="Read a debug Linux Error Log and return",
    args_schema=DebugLinuxErrorLog, 
    # coroutine= ... <- you can specify an async method if desired as well
) 

In [23]:
tools = [debug_apache_error_log, debug_zookeeper_error_log, debug_linux_error_log]
tools

[StructuredTool(name='Debug Apache Error', description='Read a debug Apache Error Log and return', args_schema=<class '__main__.DebugApacheErrorLog'>, func=<function debug_apache_error_log at 0x70be01446950>),
 StructuredTool(name='Debug Zookeeper Error', description='Read a debug Zookeeper Error Log and return', args_schema=<class '__main__.DebugZooKeeperErrorLog'>, func=<function debug_zookeeper_error_log at 0x70be0c47b010>),
 StructuredTool(name='Debug Linux Error', description='Read a debug Linux Error Log and return', args_schema=<class '__main__.DebugLinuxErrorLog'>, func=<function debug_linux_error_log at 0x70be022c0430>)]

In [24]:
system_prompt = """
You are an agent responsible for debugging error logs and mail them to Respective Mail Id.

You are provided with several functions (tools) and must choose and fill their parameters based on the given situation.

When presented with a error log, select the appropriate function to solve it.

The Functions provided to you are:

{{SUPPORTED_TOOLS_FUNCTIONS}}

Our task is to analyze the error, and provide a solution returned from the function tool.

Output format:  `Solution: <solution_returned_from_function_tool>` or a status if the operation is not supported.

Examples:

Input : Apache Error Log Format: [Time], [Level], Content - [Sun Dec 04 04:47:44 2005] [error] mod_jk child workerEnv in error state 6

Once you receive the [Event Id] and [Event Template] from tool, analyse the error log and provide a proper solution
 
Solution: <your_analysis_on_solution_returned_from_function_tool>

"""

In [25]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage

memory =  MemorySaver()
llm = ChatOpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama',
    model="llama3.2"
)  
config = {"configurable": {"thread_id": "abc123"}}

llm_agent = create_react_agent(llm, tools=tools, state_modifier=system_prompt)

In [28]:
from langchain_core.prompts import PromptTemplate
def draft_email_format(input : str, errortype:str, error:str, solution:str):
    system = """
    You are tasked with formatting the content strictly in the following format without adding extra information or explanation: 
    Extract EventId, EventTemplate, Level from the {input} and format accordingly.
    """


    e_prompt_template = PromptTemplate(
    input_variables=["input"], 
    template = system) 

    e_prompt = e_prompt_template.format(input = input)
    event = llm.invoke(e_prompt) 

    system_template = """
    You are tasked with formatting the content strictly in the following format without adding extra information or explanation:

    {errortype} Error

    {event}

    Error: {error}
    {solution}
    """

    prompt_template = PromptTemplate(
    input_variables=["event", "error", "solution"], 
    template = system_template) 


    prompt = prompt_template.format(event = event.content, errortype = errortype, error = error, solution = solution)
    result = llm.invoke(prompt) 
    return result



In [30]:
error = "[Sun Dec 04 04:51:18 2005] [error] mod_jk child workerEnv in error state 6"
errortype = "Apache"

query = f'''
{errortype} Error Log Format: [Time], [Level], Content 
{error}
'''

inputs = [HumanMessage(content=query)]

tool_llm_aimessage = llm_agent.invoke({"messages": inputs})
res = draft_email_format(tool_llm_aimessage['messages'][-2].content , errortype, error, tool_llm_aimessage['messages'][-1].content)
res

log found [Sun Dec 04 04:51:18 2005] [error] mod_jk child workerEnv in error state 6 error
inside event error [Sun Dec 04 04:51:18 2005] [error] mod_jk child workerEnv in error state 6


AIMessage(content='Apache Error\n\nEventId: E3\nEventTemplate: mod_jk child workerEnv in error state<*>\nLevel: error\n\nError: [Sun Dec 04 04:51:18 2005] [error] mod_jk child workerEnv in error state 6\nSolution: The Apache Error Log indicates that the mod_jk module is experiencing an error state with a value of 6. This could be due to a configuration issue or a problem with the Java Virtual Machine (JVM) that is being used by the web server. To resolve this issue, you should check the Apache HTTP Server configuration files to ensure that the mod_jk module is properly configured and that the JVM is running correctly. If the issue persists, you may need to consult the documentation for the specific version of the mod_jk module that is installed on your system or seek further assistance from a qualified system administrator.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 185, 'prompt_tokens': 234, 'total_tokens': 419, 'completion_tokens_de

In [32]:
print("The output is:", res.content)  

The output is: Apache Error

EventId: E3
EventTemplate: mod_jk child workerEnv in error state<*>
Level: error

Error: [Sun Dec 04 04:51:18 2005] [error] mod_jk child workerEnv in error state 6
Solution: The Apache Error Log indicates that the mod_jk module is experiencing an error state with a value of 6. This could be due to a configuration issue or a problem with the Java Virtual Machine (JVM) that is being used by the web server. To resolve this issue, you should check the Apache HTTP Server configuration files to ensure that the mod_jk module is properly configured and that the JVM is running correctly. If the issue persists, you may need to consult the documentation for the specific version of the mod_jk module that is installed on your system or seek further assistance from a qualified system administrator.


In [9]:
query = '''
Zookeeper Error Log Format: [Time], [Component], [Level], Content
2015-07-29 19:04:12,394 - INFO  [/10.10.34.11:3888:QuorumCnxManager$Listener@493] - Received connection request /10.10.34.11:45307
'''

inputs = [HumanMessage(content=query)]

tool_llm_aimessage = llm_agent.invoke({"messages": inputs})
tool_llm_aimessage
print(tool_llm_aimessage['messages'][-1].content)
print(tool_llm_aimessage['messages'][-2].content)

log found Received connection request /10.10.34.11:45307 INFO
inside event INFO Received connection request /10.10.34.11:45307
Solution: The ZooKeeper QuorumCnxManager$Listener is receiving a connection request from /10.10.34.11:45307, which is expected behavior. The INFO level indicates that this is an informational message and does not necessarily indicate an error. Therefore, no further action is required to resolve the issue.
["EventId: E50\nEventTemplate: Unexpected Exception:", "INFO", "Received connection request /10.10.34.11:45307"]


In [33]:
import base64 

def encode_byte(text):
  encoded_bytes = base64.b64encode(text.encode("utf-8"))
  return encoded_bytes

text = "Jun 16 04:16:17 combo su(pam_unix)[25548]: session opened for user news by (uid=0)"
encode_byte(text)

b'SnVuIDE2IDA0OjE2OjE3IGNvbWJvIHN1KHBhbV91bml4KVsyNTU0OF06IHNlc3Npb24gb3BlbmVkIGZvciB1c2VyIG5ld3MgYnkgKHVpZD0wKQ=='

In [10]:
query = '''
Linux Error Log Format: [Time], [Component], [Level], Content
Jun 16 04:16:17 combo su(pam_unix)[25548]: session opened for user news by (uid=0)
'''

inputs = [HumanMessage(content=query)]

tool_llm_aimessage = llm_agent.invoke({"messages": inputs})
tool_llm_aimessage
print(tool_llm_aimessage['messages'][-1].content)
print(tool_llm_aimessage['messages'][-2].content)

log found Content [Level]
inside event [Level] Content
Solution: The error log indicates a session opened for user news by (uid=0). This might be an indication of unauthorized access or an unexpected system behavior. It is recommended to investigate further and ensure that the user's session is legitimate and properly authenticated. Additionally, reviewing the system logs and authentication records can help identify any potential security vulnerabilities.
["EventId: E50\nEventTemplate: Unexpected Exception:", "[Level]", "Content"]


In [22]:
text = tool_llm_aimessage['messages'][-2].content + tool_llm_aimessage['messages'][-1].content

In [23]:
import base64

encoded_bytes = base64.b64encode(text.encode("utf-8"))
encoded_bytes

b'WyJFdmVudElkOiBFNTBcbkV2ZW50VGVtcGxhdGU6IFVuZXhwZWN0ZWQgRXhjZXB0aW9uOiIsICJbTGV2ZWxdIiwgIkNvbnRlbnQiXVNvbHV0aW9uOiBUaGUgZXJyb3IgbG9nIGluZGljYXRlcyBhIHNlc3Npb24gb3BlbmVkIGZvciB1c2VyIG5ld3MgYnkgKHVpZD0wKS4gVGhpcyBtaWdodCBiZSBhbiBpbmRpY2F0aW9uIG9mIHVuYXV0aG9yaXplZCBhY2Nlc3Mgb3IgYW4gdW5leHBlY3RlZCBzeXN0ZW0gYmVoYXZpb3IuIEl0IGlzIHJlY29tbWVuZGVkIHRvIGludmVzdGlnYXRlIGZ1cnRoZXIgYW5kIGVuc3VyZSB0aGF0IHRoZSB1c2VyJ3Mgc2Vzc2lvbiBpcyBsZWdpdGltYXRlIGFuZCBwcm9wZXJseSBhdXRoZW50aWNhdGVkLiBBZGRpdGlvbmFsbHksIHJldmlld2luZyB0aGUgc3lzdGVtIGxvZ3MgYW5kIGF1dGhlbnRpY2F0aW9uIHJlY29yZHMgY2FuIGhlbHAgaWRlbnRpZnkgYW55IHBvdGVudGlhbCBzZWN1cml0eSB2dWxuZXJhYmlsaXRpZXMu'

In [30]:
str = "WyJFdmVudElkOiBFNTBcbkV2ZW50VGVtcGxhdGU6IFVuZXhwZWN0ZWQgRXhjZXB0aW9uOiIsICJbTGV2ZWxdIiwgIkNvbnRlbnQiXVNvbHV0aW9uOiBUaGUgZXJyb3IgbG9nIGluZGljYXRlcyBhIHNlc3Npb24gb3BlbmVkIGZvciB1c2VyIG5ld3MgYnkgKHVpZD0wKS4gVGhpcyBtaWdodCBiZSBhbiBpbmRpY2F0aW9uIG9mIHVuYXV0aG9yaXplZCBhY2Nlc3Mgb3IgYW4gdW5leHBlY3RlZCBzeXN0ZW0gYmVoYXZpb3IuIEl0IGlzIHJlY29tbWVuZGVkIHRvIGludmVzdGlnYXRlIGZ1cnRoZXIgYW5kIGVuc3VyZSB0aGF0IHRoZSB1c2VyJ3Mgc2Vzc2lvbiBpcyBsZWdpdGltYXRlIGFuZCBwcm9wZXJseSBhdXRoZW50aWNhdGVkLiBBZGRpdGlvbmFsbHksIHJldmlld2luZyB0aGUgc3lzdGVtIGxvZ3MgYW5kIGF1dGhlbnRpY2F0aW9uIHJlY29yZHMgY2FuIGhlbHAgaWRlbnRpZnkgYW55IHBvdGVudGlhbCBzZWN1cml0eSB2dWxuZXJhYmlsaXRpZXMu"
base64.b64decode(str).decode("utf-8")

str

In [2]:
solution = """ 
Solution: The error log indicates a session opened for user news by (uid=0). This might be an indication of unauthorized access or an unexpected system behavior. It is recommended to investigate further and ensure that the user's session is legitimate and properly authenticated. Additionally, reviewing the system logs and authentication records can help identify any potential security vulnerabilities.
"""

error = "Jun 16 04:16:17 combo su(pam_unix)[25548]: session opened for user news by (uid=0)"

input = """ 
["EventId: E50\nEventTemplate: Unexpected Exception:", "[Level]", "Content"]
"""

In [18]:
from langchain_openai import ChatOpenAI 
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', 
    model="llama3.2"
)  

system = """
You are tasked with formatting the content strictly in the following format without adding extra information or explanation: 
Extract EventId, EventTemplate, Level from the {input} and format accordingly.
"""


e_prompt_template = PromptTemplate(
  input_variables=["input"], 
  template = system) 

e_prompt = e_prompt_template.format(input = input)
event = llm.invoke(e_prompt) 

system_template = """
You are tasked with formatting the content strictly in the following format without adding extra information or explanation:

{event}

Error: {error}
{solution}
"""

prompt_template = PromptTemplate(
  input_variables=["event", "error", "solution"], 
  template = system_template) 


prompt = prompt_template.format(event = event.content, error = error, solution = solution)
result = llm.invoke(prompt) 

print("The output is:", result.content)  

The output is: EventId: E50
EventTemplate: Unexpected Exception:
Level: [Level]
Content

Error: Jun 16 04:16:17 combo su(pam_unix)[25548]: session opened for user news by (uid=0)
 
Solution: The error log indicates a session opened for user news by (uid=0). This might be an indication of unauthorized access or an unexpected system behavior. It is recommended to investigate further and ensure that the user's session is legitimate and properly authenticated. Additionally, reviewing the system logs and authentication records can help identify any potential security vulnerabilities.


In [3]:
import uuid
str(uuid.uuid4())

'3e618d7f-2980-4c02-aed1-e3cc74748199'